# Experiment 02: Scattering + PCA + SVM






In [1]:
import sys
import random
sys.path.append('../src')
import warnings
warnings.filterwarnings("ignore") 

from utils.compute_metrics import get_metrics, get_majority_vote,log_test_metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from pprint import pprint

from itertools import product
import pickle
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt


## Feature Reduction/Selection

#### Upload Scattering Features

In [2]:
with open('../data/03_features/scattering_features_J_3.pickle', 'rb') as handle:
    scatter_dict = pickle.load(handle)
    df_scattering = scatter_dict['df']
    scattering_params = {'J':scatter_dict['J'],
                         'M':scatter_dict['M'],
                         'N':scatter_dict['N']}

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Cross Validation using SVM Classification

> Methods that exclude outliers were used to normalize the features. Patient-specific leave-one-out cross-validation (LOOCV) was applied to evaluate the classification. In each case, the test set consisted of10 images from the same patient and the training set contained 540 images from the remaining 54 patients. For each training set, fivefold cross-validation and grid search were applied to indicate the optimal SVM classifier hyperparameters and the best kernel. To address the problem of class imbalance, the SVM hyperparameter C of each class was adjusted inversely proportional to that class frequency in the training set. Label 1 indicated the image containing a fatty liver and label −1 otherwise. 


In [3]:
# Set the parameters by cross-validation
param_gamma = [1e-3, 1e-4]
param_C = [1, 10, 1000] 
svm_class_weight = [None, 'balanced']
rbf_params = list(product(['kernel'],param_gamma, param_C, svm_class_weight ))
linear_params = list(product(['linear'],param_C, svm_class_weight))
params = rbf_params + linear_params

In [4]:
def train_valid(param, X_train,X_valid,y_train, y_valid):
    if param[0] == 'kernel': 
        #The “balanced” mode uses the values of y to automatically adjust weights inversely
        #proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).
        model = SVC(gamma=param[1], C=param[2], class_weight= param[3])
    if param[0] == 'linear': 
        #The “balanced” mode uses the values of y to automatically adjust weights inversely
        #proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).
        model = LinearSVC(C=param[1], class_weight= param[2])

    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    acc, auc, specificity, sensitivity = get_metrics(y_valid, predictions)
    return acc, auc, specificity, sensitivity , predictions


In [5]:
def log_val_metrics(params, metrics, test_n_splits, num_components=5, standardize=True):
    # Important !!!! Put Correct Experiment Name
    mlflow.set_experiment('val_scattering_svm_pca_experiment')
    # log mlflow params
    for param in params:
        with mlflow.start_run():
            #log params
            mlflow.log_param('pca_n',num_components)
            mlflow.log_params(scattering_params)
            mlflow.log_param('model',f'svm: {param[0]}')
            mlflow.log_param('test K fold', test_n_splits)
            if param[0] == 'kernel':
                mlflow.log_param('gamma',param[1])
                mlflow.log_param('C',param[2])
                mlflow.log_param('class weight svm', param[3])
            if param[0] == 'linear': 
                mlflow.log_param('C',param[1])
                mlflow.log_param('class weight svm', param[2])
            #log metrics
            mlflow.log_metric('accuracy',np.array(metrics[str(param)]['acc']).mean())
            mlflow.log_metric('AUC',np.array(metrics[str(param)]['auc']).mean())
            mlflow.log_metric('specificity',np.array(metrics[str(param)]['specificity']).mean())
            mlflow.log_metric('sensitivity',np.array(metrics[str(param)]['sensitivity']).mean())
    print("Done logging validation params in MLFlow")

In [6]:
df = df_scattering
pca_n_components = 5
standardize = True
test_metrics={}  
#majority vote results
test_metrics_mv={} 
# test split 55 et 11
test_n_splits = 11
group_kfold_test = GroupKFold(n_splits=test_n_splits)
seed= 11
df_pid = df['id']
df_y = df['class']
fold_c =1 

for train_index, test_index in group_kfold_test.split(df, 
                                                  df_y, 
                                                  df_pid):
    random.seed(seed)
    random.shuffle(train_index)
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = df_y.iloc[train_index], df_y.iloc[test_index]
    
    X_test = X_test.drop(columns=['id', 'class'])
    X_train_pid = X_train.pop('id')
    X_train = X_train.drop(columns=['class'])
    
    # Do cross-validation for hyperparam tuning
    group_kfold_val = GroupKFold(n_splits=5)
    metrics={}
    #X_train_y = df.pop('class')
    for subtrain_index, valid_index in group_kfold_val.split(X_train, 
                                                      y_train, 
                                                      X_train_pid):
                                   
        X_subtrain, X_valid = X_train.iloc[subtrain_index], X_train.iloc[valid_index]
        y_subtrain, y_valid = y_train.iloc[subtrain_index], y_train.iloc[valid_index]


        
        pca = PCA(n_components=pca_n_components,random_state = seed)           
        X_subtrain = pca.fit_transform(X_subtrain)
        X_valid = pca.transform(X_valid)
        
        #standardize
        if standardize:
            scaler = StandardScaler()
            X_subtrain = scaler.fit_transform(X_subtrain)
            X_valid = scaler.transform(X_valid)
        

        for param in tqdm(params):
            if str(param) not in metrics.keys() :
                metrics[str(param)] ={'acc':[], 'auc':[], 'sensitivity':[], 'specificity':[]}
                                   
            acc, auc, specificity, sensitivity,_ = train_valid(param, X_subtrain,X_valid,y_subtrain, y_valid)
            metrics[str(param)]['auc'].append(auc)
            metrics[str(param)]['acc'].append(acc)
            metrics[str(param)]['sensitivity'].append(sensitivity)
            metrics[str(param)]['specificity'].append(specificity)
    #log validation metrics for all combination of params
    log_val_metrics(params, metrics, test_n_splits, pca_n_components, standardize=standardize )
    
    #highest accuracy
    index_param_max = np.array([np.array(metrics[str(param)]['auc']).mean() for param in params]).argmax()
    print('From all the combinations, the highest accuracy was achieved with', params[index_param_max])
    #train and test with max param

    
    pca = PCA(n_components=pca_n_components)           
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    #standardize
    if standardize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    acc, auc, specificity, sensitivity, predictions = train_valid(params[index_param_max], X_train, X_test, y_train, y_test)
    
    #compute majority vote metrics
    acc_mv, auc_mv, specificity_mv, sensitivity_mv = get_majority_vote(y_test, predictions)
    
    print('FOLD '+ str(fold_c) + ':  acc ' + str(acc) +  ', auc ' +  str(auc) +  ', specificity '+ str(specificity)
          + ', sensitivity ' + str(sensitivity))
    print('FOLD '+ str(fold_c) + ':  MV acc ' + str(acc_mv) +  ', MV auc ' +  str(auc_mv) +  ', MV specificity '+ str(specificity_mv)
          + ', MV sensitivity ' + str(sensitivity_mv))
    
    test_metrics[fold_c]=  {'acc':acc, 'auc':auc, 'sensitivity':sensitivity, 'specificity':specificity, 'param':params[index_param_max]}
    test_metrics_mv[fold_c]=  {'acc':acc_mv, 'auc':auc_mv, 'sensitivity':sensitivity_mv, 'specificity':specificity_mv, 'param':params[index_param_max]}
    
    fold_c +=1 

log_test_metrics(test_metrics, test_metrics_mv, test_n_splits, 'Scattering features + PCA + SVM',None, seed, pca_n_components, standardize=standardize)


100%|██████████| 18/18 [00:00<00:00, 110.99it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 1, 'balanced')
FOLD 1:  acc 1.0, auc 1.0, specificity 1.0, sensitivity 1.0
FOLD 1:  MV acc 1.0, MV auc 1.0, MV specificity 1.0, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 109.26it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 1000, 'balanced')
FOLD 2:  acc 0.86, auc 0.8833333333333333, specificity 0.7666666666666667, sensitivity 1.0
FOLD 2:  MV acc 0.8, MV auc 0.8333333333333334, MV specificity 0.6666666666666666, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 120.26it/s]


...

Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 1, 'balanced')
FOLD 9:  acc 0.6, auc 0.75, specificity 1.0, sensitivity 0.5
FOLD 9:  MV acc 0.6, MV auc 0.75, MV specificity 1.0, MV sensitivity 0.5


100%|██████████| 18/18 [00:00<00:00, 135.56it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 10, 'balanced')
FOLD 10:  acc 0.4, auc 0.25, specificity 0.0, sensitivity 0.5
FOLD 10:  MV acc 0.4, MV auc 0.25, MV specificity 0.0, MV sensitivity 0.5


100%|██████████| 18/18 [00:00<00:00, 117.02it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 10, 'balanced')
FOLD 11:  acc 0.8, auc 0.75, specificity 1.0, sensitivity 0.5
FOLD 11:  MV acc 0.8, MV auc 0.75, MV specificity 1.0, MV sensitivity 0.5
0.8054545454545454 0.7810606060606061
0.8 0.7765151515151516
Experiment done


In [7]:
df = df_scattering
pca_n_components = 8
standardize = True
test_metrics={}  
#majority vote results
test_metrics_mv={} 
# test split 55 et 11
test_n_splits = 11
group_kfold_test = GroupKFold(n_splits=test_n_splits)
seed= 11
df_pid = df['id']
df_y = df['class']
fold_c =1 

for train_index, test_index in group_kfold_test.split(df, 
                                                  df_y, 
                                                  df_pid):
    random.seed(seed)
    random.shuffle(train_index)
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = df_y.iloc[train_index], df_y.iloc[test_index]
    
    X_test = X_test.drop(columns=['id', 'class'])
    X_train_pid = X_train.pop('id')
    X_train = X_train.drop(columns=['class'])
    
    # Do cross-validation for hyperparam tuning
    group_kfold_val = GroupKFold(n_splits=5)
    metrics={}
    #X_train_y = df.pop('class')
    for subtrain_index, valid_index in group_kfold_val.split(X_train, 
                                                      y_train, 
                                                      X_train_pid):
                                   
        X_subtrain, X_valid = X_train.iloc[subtrain_index], X_train.iloc[valid_index]
        y_subtrain, y_valid = y_train.iloc[subtrain_index], y_train.iloc[valid_index]


        
        pca = PCA(n_components=pca_n_components,random_state = seed)           
        X_subtrain = pca.fit_transform(X_subtrain)
        X_valid = pca.transform(X_valid)
        
        #standardize
        if standardize:
            scaler = StandardScaler()
            X_subtrain = scaler.fit_transform(X_subtrain)
            X_valid = scaler.transform(X_valid)
        

        for param in tqdm(params):
            if str(param) not in metrics.keys() :
                metrics[str(param)] ={'acc':[], 'auc':[], 'sensitivity':[], 'specificity':[]}
                                   
            acc, auc, specificity, sensitivity,_ = train_valid(param, X_subtrain,X_valid,y_subtrain, y_valid)
            metrics[str(param)]['auc'].append(auc)
            metrics[str(param)]['acc'].append(acc)
            metrics[str(param)]['sensitivity'].append(sensitivity)
            metrics[str(param)]['specificity'].append(specificity)
    #log validation metrics for all combination of params
    log_val_metrics(params, metrics, test_n_splits, pca_n_components, standardize=standardize )
    
    #highest accuracy
    index_param_max = np.array([np.array(metrics[str(param)]['auc']).mean() for param in params]).argmax()
    print('From all the combinations, the highest accuracy was achieved with', params[index_param_max])
    #train and test with max param

    
    pca = PCA(n_components=pca_n_components)           
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    #standardize
    if standardize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    acc, auc, specificity, sensitivity, predictions = train_valid(params[index_param_max], X_train, X_test, y_train, y_test)
    
    #compute majority vote metrics
    acc_mv, auc_mv, specificity_mv, sensitivity_mv = get_majority_vote(y_test, predictions)
    
    print('FOLD '+ str(fold_c) + ':  acc ' + str(acc) +  ', auc ' +  str(auc) +  ', specificity '+ str(specificity)
          + ', sensitivity ' + str(sensitivity))
    print('FOLD '+ str(fold_c) + ':  MV acc ' + str(acc_mv) +  ', MV auc ' +  str(auc_mv) +  ', MV specificity '+ str(specificity_mv)
          + ', MV sensitivity ' + str(sensitivity_mv))
    
    test_metrics[fold_c]=  {'acc':acc, 'auc':auc, 'sensitivity':sensitivity, 'specificity':specificity, 'param':params[index_param_max]}
    test_metrics_mv[fold_c]=  {'acc':acc_mv, 'auc':auc_mv, 'sensitivity':sensitivity_mv, 'specificity':specificity_mv, 'param':params[index_param_max]}
    
    fold_c +=1 

log_test_metrics(test_metrics, test_metrics_mv, test_n_splits, 'Scattering features + PCA + SVM',None, seed, pca_n_components, standardize=standardize)


/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 18/18 [00:00<00:00, 76.17it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 1, 'balanced')
FOLD 1:  acc 0.8, auc 0.8333333333333333, specificity 1.0, sensitivity 0.6666666666666666
FOLD 1:  MV acc 0.8, MV auc 0.8333333333333333, MV specificity 1.0, MV sensitivity 0.6666666666666666


100%|██████████| 18/18 [00:00<00:00, 85.39it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 1000, 'balanced')
FOLD 2:  acc 0.84, auc 0.8666666666666667, specificity 0.7333333333333333, sensitivity 1.0
FOLD 2:  MV acc 0.8, MV auc 0.8333333333333334, MV specificity 0.6666666666666666, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 89.27it/s]


...

Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 10, 'balanced')
FOLD 9:  acc 0.6, auc 0.75, specificity 1.0, sensitivity 0.5
FOLD 9:  MV acc 0.6, MV auc 0.75, MV specificity 1.0, MV sensitivity 0.5


100%|██████████| 18/18 [00:00<00:00, 88.85it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 1, 'balanced')
FOLD 10:  acc 0.4, auc 0.25, specificity 0.0, sensitivity 0.5
FOLD 10:  MV acc 0.4, MV auc 0.25, MV specificity 0.0, MV sensitivity 0.5


100%|██████████| 18/18 [00:00<00:00, 45.65it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 10, 'balanced')
FOLD 11:  acc 0.8, auc 0.75, specificity 1.0, sensitivity 0.5
FOLD 11:  MV acc 0.8, MV auc 0.75, MV specificity 1.0, MV sensitivity 0.5
0.790909090909091 0.7765151515151516
0.7818181818181817 0.7689393939393938
Experiment done


In [ ]:
df = df_scattering
pca_n_components = 7
standardize = True
test_metrics={}  
#majority vote results
test_metrics_mv={} 
# test split 55 et 11
test_n_splits = 11
group_kfold_test = GroupKFold(n_splits=test_n_splits)
seed= 11
df_pid = df['id']
df_y = df['class']
fold_c =1 

for train_index, test_index in group_kfold_test.split(df, 
                                                  df_y, 
                                                  df_pid):
    random.seed(seed)
    random.shuffle(train_index)
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = df_y.iloc[train_index], df_y.iloc[test_index]
    
    X_test = X_test.drop(columns=['id', 'class'])
    X_train_pid = X_train.pop('id')
    X_train = X_train.drop(columns=['class'])
    
    # Do cross-validation for hyperparam tuning
    group_kfold_val = GroupKFold(n_splits=5)
    metrics={}
    #X_train_y = df.pop('class')
    for subtrain_index, valid_index in group_kfold_val.split(X_train, 
                                                      y_train, 
                                                      X_train_pid):
                                   
        X_subtrain, X_valid = X_train.iloc[subtrain_index], X_train.iloc[valid_index]
        y_subtrain, y_valid = y_train.iloc[subtrain_index], y_train.iloc[valid_index]


        
        pca = PCA(n_components=pca_n_components,random_state = seed)           
        X_subtrain = pca.fit_transform(X_subtrain)
        X_valid = pca.transform(X_valid)
        
        #standardize
        if standardize:
            scaler = StandardScaler()
            X_subtrain = scaler.fit_transform(X_subtrain)
            X_valid = scaler.transform(X_valid)
        

        for param in tqdm(params):
            if str(param) not in metrics.keys() :
                metrics[str(param)] ={'acc':[], 'auc':[], 'sensitivity':[], 'specificity':[]}
                                   
            acc, auc, specificity, sensitivity,_ = train_valid(param, X_subtrain,X_valid,y_subtrain, y_valid)
            metrics[str(param)]['auc'].append(auc)
            metrics[str(param)]['acc'].append(acc)
            metrics[str(param)]['sensitivity'].append(sensitivity)
            metrics[str(param)]['specificity'].append(specificity)
    #log validation metrics for all combination of params
    log_val_metrics(params, metrics, test_n_splits, pca_n_components, standardize=standardize )
    
    #highest accuracy
    index_param_max = np.array([np.array(metrics[str(param)]['auc']).mean() for param in params]).argmax()
    print('From all the combinations, the highest accuracy was achieved with', params[index_param_max])
    #train and test with max param

    
    pca = PCA(n_components=pca_n_components)           
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    #standardize
    if standardize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    acc, auc, specificity, sensitivity, predictions = train_valid(params[index_param_max], X_train, X_test, y_train, y_test)
    
    #compute majority vote metrics
    acc_mv, auc_mv, specificity_mv, sensitivity_mv = get_majority_vote(y_test, predictions)
    
    print('FOLD '+ str(fold_c) + ':  acc ' + str(acc) +  ', auc ' +  str(auc) +  ', specificity '+ str(specificity)
          + ', sensitivity ' + str(sensitivity))
    print('FOLD '+ str(fold_c) + ':  MV acc ' + str(acc_mv) +  ', MV auc ' +  str(auc_mv) +  ', MV specificity '+ str(specificity_mv)
          + ', MV sensitivity ' + str(sensitivity_mv))
    
    test_metrics[fold_c]=  {'acc':acc, 'auc':auc, 'sensitivity':sensitivity, 'specificity':specificity, 'param':params[index_param_max]}
    test_metrics_mv[fold_c]=  {'acc':acc_mv, 'auc':auc_mv, 'sensitivity':sensitivity_mv, 'specificity':specificity_mv, 'param':params[index_param_max]}
    
    fold_c +=1 

log_test_metrics(test_metrics, test_metrics_mv, test_n_splits, 'Scattering features + PCA + SVM',None, seed, pca_n_components, standardize=standardize)


/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 18/18 [00:00<00:00, 123.63it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 10, 'balanced')
FOLD 1:  acc 1.0, auc 1.0, specificity 1.0, sensitivity 1.0
FOLD 1:  MV acc 1.0, MV auc 1.0, MV specificity 1.0, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 118.65it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 1000, 'balanced')
FOLD 2:  acc 0.86, auc 0.8833333333333333, specificity 0.7666666666666667, sensitivity 1.0
FOLD 2:  MV acc 0.8, MV auc 0.8333333333333334, MV specificity 0.6666666666666666, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 51.30it/s]


...

Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.001, 10, 'balanced')
FOLD 8:  acc 1.0, auc 1.0, specificity 1.0, sensitivity 1.0
FOLD 8:  MV acc 1.0, MV auc 1.0, MV specificity 1.0, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 133.99it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 1000, None)
FOLD 9:  acc 0.8, auc 0.5, specificity 0.0, sensitivity 1.0
FOLD 9:  MV acc 0.8, MV auc 0.5, MV specificity 0.0, MV sensitivity 1.0


100%|██████████| 18/18 [00:00<00:00, 161.15it/s]


Done logging validation params in MLFlow
From all the combinations, the highest accuracy was achieved with ('kernel', 0.0001, 10, 'balanced')
